In [2]:
import cv2
import sys, math, time, random
from datetime import datetime
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt



In [3]:
def sigmoid(gamma):     #convert to probability
	if gamma < 0:
		return 1 - 1/(1 + math.exp(gamma/5))
	else:
		return 1/(1 + math.exp(-gamma/5))

def Vfunction(gamma):
	# return abs(np.tanh(gamma))

	#another V-shaped function
	val=(math.pi/2)*gamma
	val=np.arctan(val)
	val=(2/math.pi)*val
	return abs(val)

def initialize(popSize,dimension):
	population = np.zeros((popSize,dimension))
	minn = 0
	maxx = 255
	for i in range(popSize):
		for j in range(dimension):
			random.seed( i + j + time.time() )
			population[i][j] = random.randint(minn,maxx)
		population[i].sort()
		population[i][0] = 0
		population[i][dimension-1] = 255
	return population




In [4]:
def fitness(agentCurr,inputAgent,inputImage,iterNo):
	# fitness
	# laplacian = cv2.Laplacian(img,cv2.CV_64F)
	img = deepcopy(inputImage)
	img2 = transformImage(agentCurr,inputAgent,inputImage,iterNo)
	edges = cv2.Canny(img2,100,200)
	count=0
	intsum=1.1      
	tmp=deepcopy(img2)
	dimensions=img2.shape
	for r in range(0,dimensions[0]):
		for c in range(0,dimensions[1]):
			if (edges[r,c]==255):
				count+=1
				tmp[r,c]=-1
				intsum+=img2[r,c]
	
	#entropy of image
	flatImg =  [x for sublist in img2 for x in sublist]
	uniqImg = set(flatImg)
	Hx = 0
	for x in uniqImg:
		p_x = flatImg.count(x) / len(uniqImg)
		Hx += ((- x) * (math.log2(p_x)))
	#Well, How about calling the in built entropy function?
	#Yep'
	# will try that one if this disappoints me

	# print('sum = ',sum,end=' ')
	# print('log(sum) = ',math.log(sum))
	stepSize = 5
	localcontrast=1.1
	(w_width, w_height) = (5, 5)
	
	for x in range(0, tmp.shape[1] - w_width , stepSize):
		for y in range(0, tmp.shape[0] - w_height, stepSize):
			tmp2 = tmp[x:x + w_width, y:y + w_height]
			localcontrast+=((max(tmp2.flatten()[~np.isin(tmp2.flatten(),-1)]))-(min(tmp2.flatten()[~np.isin(tmp2.flatten(),-1)])))
	

			fit=math.log(math.log(intsum))*Hx*(count)*(1.02**(-(np.mean(img2)-np.mean(img))**2))*((max(img2.flatten())-min(img2.flatten()))**2)*((localcontrast))    
	return fit




In [17]:
def allfit(population,inputAgent,inputImage,iterNo):
	x=np.shape(population)[0]
	acc=np.zeros(x)
	for i in range(x):
		acc[i]=fitness(population[i],inputAgent,inputImage,iterNo)     
		print(acc[i])
	return acc



In [13]:
def transformImage(currAgent,inputAgent,inputImage,iterNo):
	tarnsImage = inputImage.copy()
	row = np.shape(inputImage)[0]
	col = np.shape(inputImage)[1]
	for i in range(row):
		for j in range(col):
			k = inputAgent.index(tarnsImage[i][j])
			tarnsImage[i][j] = currAgent[k]
	# cv2.imwrite("intermediate/"+str(iterNo)+'_'+str(agentNo)+'.png',tarnsImage)
	# cv2.imshow('new image',tarnsImage)
	# cv2.waitKey(0)
	# cv2.destroyAllWindows()
	return tarnsImage


In [14]:
popSize = 5
maxIter = 5
imageName = 'InputImages/kodakDataset/kodim01.png'

In [ ]:
img = cv2.imread(imageName,0)
print(np.shape(img))

flatImg =  [x for sublist in img for x in sublist]
# print(flatImg)
uniqImg = set(flatImg)
agentLength = len(uniqImg)
agentImg = list(uniqImg)
agentImg.sort()
print(agentImg)

x_axis = []
y_axis = []

population = initialize(popSize,agentLength)
# print(population)
dimension = agentLength

start_time = datetime.now()
fitList = allfit(population,agentImg,img,0)
temp = [-x for x in fitList]
#sort agents
population = [x for _,x in sorted(zip(temp,population))]
fitList.sort()
fitList.reverse()



(512, 768)
[81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174]


In [ ]:
vals = im.mean(axis=2).flatten()
# plot histogram with 255 bins
b, bins, patches = plt.hist(vals, 255)
plt.xlim([0,255])
plt.show()

In [ ]:
counts, bins = np.histogram(data)

In [18]:

receivedList = []
while currIter in range(1,maxIter):
    area = int(0.5 * popSize)
    random.seed(time.time() + 10 )
    parent1 = random.randint(0,popSize-1)
    random.seed(time.time() + 19 )
    parent2 = random.randint(0,popSize-1)
    while(parent2 == parent1 and parent2 in receivedList):
        parent2 = random.randint(0,popSize-1)
    receivedList.append(parent2)
    print(fitList[parent1],fitList[parent2])
    
    random.seed(time.time() + 29 )
    offspring = np.multiply(random.random(),population[parent2])
    if abs(parent1 - parent2 )<= area:
        p = random.uniform() # we can make this ratio based on their fitness
        q = (1 - p)
        offspring = np.add(np.multiply(p,population[parent1]),np.multiply(q,population[parent2]))

    currFit = fitness(offspring,inputAgent,inputImage,currIter)
    inx = next(idx for idx, value in enumerate(fitList) if value < currFit)	
    if inx<popSize:
        np.insert(population,inx,offspring,axis=0)
        print('after inserting offspring',np.shape(population))
        np.delete(population,popSize-1,axis=0)
        print('after deleting offspring',np.shape(population))
        receivedList.remove(inx)
        fitList.insert(inx,currFit)
bestAgent = population[0].copy()
bestImage = transformImage(bestAgent,inputAgent,inputImage,maxIter)
transImageName = imageName.split('/')[-1]
transImageName = "t_"+transImageName
cv2.imwrite("output/"+transImageName,tarnsImage)

histr = cv2.calcHist([outputImage],[0],None,[256],[0,256]) 
plt.plot(histr)
plt.show()

(512, 768)


KeyboardInterrupt: 